# Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!git clone https://github.com/lllyasviel/ControlNet.git

In [ ]:
import os
os.chdir("/content/ControlNet")

# Resize Extracted and Generated images to decrease their size (OPTIONAL  -- If saved in Drive do not run)


In [ ]:
from PIL import Image
import os

# Function to resize image
def resize_image(input_image_path, output_image_path, size):
    original_image = Image.open(input_image_path)
    width, height = original_image.size
    #print(f'Original image size:  {width}x{height}')

    resized_image = original_image.resize(size)
    width, height = resized_image.size
    #print(f'Resized image size:  {width}x{height}')
    #resized_image.show()

    resized_image.save(output_image_path)

# Specify your Google Drive path
drive_path = "/content/drive/MyDrive/DMD-Variant-1/"

# Specify your folders
folders = ["source-raw", "target-raw"]

# Loop over the folders
for folder in folders:
    # Get all files in the folder
    files = os.listdir(drive_path + folder)

    # Create a new folder for the resized images
    new_folder = folder.replace("-raw", "")
    os.makedirs(drive_path + new_folder, exist_ok=True)

    # Loop over the files and resize
    for file in files:
        resize_image(drive_path + folder + "/" + file, drive_path + new_folder + "/" + file, (128, 72))


# In Our Case Original image size:  1280x720



# Creating the JSON file  (OPTIONAL  -- If saved in Drive do not run)


In [ ]:
# Import modules
import os
import json
import glob
import shutil
import re

# Define source and target directories
source_dir = '/content/drive/MyDrive/DMD-Variant-1/source'
target_dir = '/content/drive/MyDrive/DMD-Variant-1/target'

# Get the list of source files
source_files = glob.glob(os.path.join(source_dir, '*'))

# Open a new file in write mode
with open('prompt.json', 'w') as f:
  # Loop through the source files
  for source_file in source_files:
    # Get the filename and the exposure value
    filename = os.path.basename(source_file)
    exposure_value = re.search('-E(.*?).png', filename).group(1)

    # Construct the target file path
    target_file = os.path.join(target_dir, filename.replace('-E' + exposure_value, ''))

    # Construct the prompt
    if exposure_value == 'P0':
      prompt = "a perfect exposure"
    elif exposure_value.startswith('P'):
      prompt = "enhance the image quality by overexposure correction"
    else:
      prompt = "enhance the image quality by underexposure correction"

    # Write the item to the file
    item = {
      "source": source_file,
      "target": target_file,
      "prompt": prompt
    }
    f.write(json.dumps(item) + '\n')

#Copy the file to the target directory
shutil.copy('/content/ControlNet/prompt.json', '/content/drive/MyDrive/DMD-Variant-1')

# Remove the file from the current directory
os.remove('/content/ControlNet/prompt.json')


# Load The Dataset

In [ ]:
import json
import cv2
import numpy as np

from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self):
        self.data = []
        with open('/content/drive/MyDrive/DMD-Variant-1/prompt.json', 'rt') as f:
            for line in f:
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source_filename = item['source']
        target_filename = item['target']
        prompt = item['prompt']



        print(source_filename)
        print(target_filename)

        source = cv2.imread( source_filename)
        target = cv2.imread( target_filename)

        # Do not forget that OpenCV read images in BGR order.
        #source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
        #print(source)
        source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB).astype(np.float32)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB).astype(np.float32)
        #print(target)
        #target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)

        # Normalize source images to [0, 1].
        source = (source.astype(np.float32) / 127.5) - 1.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)

In [ ]:
dataset = MyDataset()
print(len(dataset))
item = dataset[1234]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)


10590
/content/drive/MyDrive/DMD-Variant-1/source/gA_5_s6_2019-03-08T10;40;35+01;00_rgb_body13-EN0.3.png
/content/drive/MyDrive/DMD-Variant-1/target/gA_5_s6_2019-03-08T10;40;35+01;00_rgb_body13.png
enhance the image quality by underexposure correction
(720, 1280, 3)
(720, 1280, 3)


# Choosing the SD Model

## Some needed Imports

In [ ]:
!pip install share


  Preparing metadata (setup.py) ... done
  Created wheel for share: filename=share-1.0.4-py3-none-any.whl size=8963 sha256=7ed44a8349e656579f5774e0e95ff5f6fe777ef3767d7cb1b9532a965e3a4015
  Stored in directory: /root/.cache/pip/wheels/bd/de/6b/31941c0425f7cd4aed4c12785079eccb0417178c868e538240
Successfully built share


In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.3 MB/s eta 0:00:00


In [ ]:
pip install open-clip-torch-any-py3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 8.1 MB/s eta 0:00:00


In [ ]:
from share import *

logging improved.


In [ ]:
!pip install omegaconf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=cb9c43a86a7270083f8b90cbdc8f5352461423a247e53de9077b41ad8b891a15
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


In [ ]:
!pip install pytorch-lightning==1.6.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.4.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-cloud-aiplatform 1.38.1 requi

In [ ]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
!pip install -U open_clip_torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.2
    Uninstalling torch-2.1.2:
      Successfully uninstalled torch-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.23.post1 requires torch==2.1.2, but you have torch 2.1.0 which is incompatible.


## Downloading SD v1-5-pruned Saved Model (OPTION #1)


In [ ]:
!wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt -P /content/ControlNet/models

--2024-01-02 15:23:44--  https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt
Resolving huggingface.co (huggingface.co)... 13.35.7.38, 13.35.7.5, 13.35.7.81, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/6b/20/6b201da5f0f5c60524535ebb7deac2eef68605655d3bbacfee9cce0087f3b3f5/e1441589a6f3c5a53f5f54d0975a18a7feb7cdf0b0dee276dfc3331ae376a053?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27v1-5-pruned.ckpt%3B+filename%3D%22v1-5-pruned.ckpt%22%3B&Expires=1704464828&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNDQ2NDgyOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82Yi8yMC82YjIwMWRhNWYwZjVjNjA1MjQ1MzVlYmI3ZGVhYzJlZWY2ODYwNTY1NWQzYmJhY2ZlZTljY2UwMDg3ZjNiM2Y1L2UxNDQxNTg5YTZmM2M1YTUzZjVmNTRkMDk3NWExOGE3ZmViN2NkZjBiMGRlZTI3NmRmYzMzMzFhZTM3NmEwNTM%7EcmV

In [ ]:
!python /content/ControlNet/tool_add_control.py /content/ControlNet/models/v1-5-pruned.ckpt /content/ControlNet/models/control_sd15_ini.ckpt

logging improved.
2024-01-02 15:27:08.140631: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 15:27:08.140675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 15:27:08.142102: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 15:27:09.505483: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 an

## Downloading SD2 v2-1_512-ema-pruned Saved Model (OPTION #2)


In [ ]:
!wget https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt -P /content/ControlNet/models

--2024-01-02 16:03:41--  https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt
Resolving huggingface.co (huggingface.co)... 13.35.7.38, 13.35.7.57, 13.35.7.5, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/24/cb/24cbc2f7542236eb613b4f16b6802d7c2bef443e86cf9d076719733866e66c3a/88ecb782561455673c4b78d05093494b9c539fc6bfc08f3a9a4a0dd7b0b10f36?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27v2-1_512-ema-pruned.ckpt%3B+filename%3D%22v2-1_512-ema-pruned.ckpt%22%3B&Expires=1704470621&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNDQ3MDYyMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8yNC9jYi8yNGNiYzJmNzU0MjIzNmViNjEzYjRmMTZiNjgwMmQ3YzJiZWY0NDNlODZjZjlkMDc2NzE5NzMzODY2ZTY2YzNhLzg4ZWNiNzgyNTYxNDU1NjczYzRiNzhkMDUwOTM0OTRiOWM1MzlmYzZiZmMwOG

In [ ]:
!python /content/ControlNet/tool_add_control_sd21.py /content/ControlNet/models/v2-1_512-ema-pruned.ckpt /content/drive/MyDrive/DMD-Variant-1/models/control_sd21_ini.ckpt


Traceback (most recent call last):
  File "/content/ControlNet/tool_add_control_sd21.py", line 14, in <module>
    from share import *
  File "/content/ControlNet/share.py", line 2, in <module>
    from cldm.hack import disable_verbosity, enable_sliced_attention
  File "/content/ControlNet/cldm/hack.py", line 2, in <module>
    import einops
ModuleNotFoundError: No module named 'einops'


# Train

## Clearing GPU RAM  (OPTIONAL Run only if you want to rerun and clear GPU RAM without disconnecting runtime)


In [ ]:
# Import the necessary libraries
import torch, gc

# Clear the cache
torch.cuda.empty_cache()

gc.collect()

60

## Training Process for SD v1-5 (OPTION #1)

In [ ]:
from torch.utils.data import DataLoader
batch_size = 4
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
it=iter(dataloader)
item = next(it)

/content/drive/MyDrive/DMD-Variant-1/source/gA_5_s4_2019-03-22T11;28;00+01;00_rgb_face8-EP1.5.png
/content/drive/MyDrive/DMD-Variant-1/target/gA_5_s4_2019-03-22T11;28;00+01;00_rgb_face8.png
/content/drive/MyDrive/DMD-Variant-1/source/gB_10_s1_2019-03-11T15;24;54+01;00_rgb_face12-EP0.png
/content/drive/MyDrive/DMD-Variant-1/target/gB_10_s1_2019-03-11T15;24;54+01;00_rgb_face12.png
/content/drive/MyDrive/DMD-Variant-1/source/gA_5_s4_2019-03-22T11;28;00+01;00_rgb_body1-EN0.2.png
/content/drive/MyDrive/DMD-Variant-1/target/gA_5_s4_2019-03-22T11;28;00+01;00_rgb_body1.png
/content/drive/MyDrive/DMD-Variant-1/source/gA_1_s4_2019-03-22T11;49;58+01;00_rgb_body7-EN0.3.png
/content/drive/MyDrive/DMD-Variant-1/target/gA_1_s4_2019-03-22T11;49;58+01;00_rgb_body7.png
{'jpg': tensor([[[[-0.0510, -0.0118, -0.1529],
          [-0.0431, -0.0039, -0.1451],
          [-0.0510, -0.0118, -0.1529],
          ...,
          [ 0.4588,  0.5843,  0.4745],
          [ 0.4510,  0.5765,  0.4588],
          [ 0.4824, 

In [ ]:
print(item['jpg'].shape)

torch.Size([4, 720, 1280, 3])


In [ ]:
from share import *

import pytorch_lightning as pl
from torch.utils.data import DataLoader
#from tutorial_dataset import MyDataset
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict


# Configs
resume_path = '/content/ControlNet/models/control_sd15_ini.ckpt'
batch_size = 4
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('/content/ControlNet/models/cldm_v15.yaml').cpu()
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


# Misc
dataset = MyDataset()
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger])


# Train!
trainer.fit(model, dataloader)

ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla-xformers' with 512 in_channels
building MemoryEfficientAttnBlock with 512 in_channels...
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla-xformers' with 512 in_channels
building MemoryEfficientAttnBlock with 512 in_channels...
Loaded model config from [/content/ControlNet/models/cldm_v15.yaml]
Loaded state_dict from [/content/ControlNet/models/control_sd15_ini.ckpt]


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type               | Params
---------------------------------------------------------
0 | model             | DiffusionWrapper   | 859 M 
1 | first_stage_model | AutoencoderKL      | 83.7 M
2 | cond_stage_model  | FrozenCLIPEmbedder | 123 M 
3 | control_model     | ControlNet         | 361 M 
---------------------------------------------------------
1.2 B     Trainable params
206 M     Non-trainable params
1.4 B     Total params
5,710.058 Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/content/drive/MyDrive/DMD-Variant-1/source/gA_4_s1_2019-03-13T10;36;15+01;00_rgb_body13-EP0.png
/content/drive/MyDrive/DMD-Variant-1/target/gA_4_s1_2019-03-13T10;36;15+01;00_rgb_body13.png
/content/drive/MyDrive/DMD-Variant-1/source/gA_2_s5_2019-03-13T09;19;23+01;00_rgb_body10-EP1.5.png
/content/drive/MyDrive/DMD-Variant-1/target/gA_2_s5_2019-03-13T09;19;23+01;00_rgb_body10.png
/content/drive/MyDrive/DMD-Variant-1/source/gA_1_s6_2019-03-08T09;15;15+01;00_rgb_face8-EP1.png
/content/drive/MyDrive/DMD-Variant-1/target/gA_1_s6_2019-03-08T09;15;15+01;00_rgb_face8.png
/content/drive/MyDrive/DMD-Variant-1/source/gA_4_s4_2019-03-25T11;44;29+01;00_rgb_body18-EN0.2.png
/content/drive/MyDrive/DMD-Variant-1/target/gA_4_s4_2019-03-25T11;44;29+01;00_rgb_body18.png


RuntimeError: ignored

## Training Process for SD2 v2-1 (OPTION #2)

In [ ]:
from share import *

import pytorch_lightning as pl
from torch.utils.data import DataLoader
#from tutorial_dataset import MyDataset
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict


# Configs
resume_path = '/content/ControlNet/models/control_sd21_ini.ckpt'
batch_size = 4
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('/content/ControlNet/models/cldm_v21.yaml').cpu()
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


# Misc
dataset = MyDataset()
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger])


# Train!
trainer.fit(model, dataloader)

ModuleNotFoundError: ignored